# Value Iteration on FrozenLake-v1 (deterministic/non-deterministic)


In [1]:
#file: frozen_VALUE_ITERATION.ipynb
import numpy as np
import time
try:
    import gymnasium as gym
except ImportError:
    import gym


In [2]:
# Custom FrozenLake map (same as policy-iteration notebook)
custom_map = [
    'SFFHFF',
    'HFFFFF',
    'FFHFFF',
    'HHFHHF',
    'FFFGFF'
]
map_desc = custom_map


In [3]:
# Create env with human rendering so a window appears when stepping
env = gym.make(
    'FrozenLake-v1',
    is_slippery=False,# True for stochastic environment
    desc=map_desc,
    render_mode='human'
)


In [4]:
# Access MDP transitions and spaces
P = (env.unwrapped.P).copy()  # dict: P[s][a] -> list of (prob, next_state, reward, terminated)
nS = env.observation_space.n
nA = env.action_space.n

# Hyperparameters
gamma = 0.99
theta = 1e-8

print(f'Number of states: {nS}, Number of actions: {nA}')
#print (f'P= {P}')

Number of states: 30, Number of actions: 4


P is a dictionary describing the Markov Decision Process (MDP) transitions.

- P[s][a] gives a list of possible transitions from state s when taking action a.
- Each element in P[s][a] is a tuple: (probability, next_state, reward, terminated).

In [5]:
def value_iteration(gamma: float = gamma, theta: float = theta):
    """
    Compute optimal value function V_* via Bellman optimality updates,
    then extract a greedy policy. Returns (pi, V, iterations).
    """
    V = np.zeros(nS, dtype=np.float64)
    iterations = 0
    while True:
        delta = 0.0
        iterations += 1
        for s in range(nS):
            v_old = V[s]
            q = np.zeros(nA, dtype=np.float64)
            for a in range(nA):
                for (prob, ns, r, done) in P[s][a]:
                    q[a] += prob * (r + gamma * (0.0 if done else V[ns]))
            V[s] = np.max(q)
            delta = max(delta, abs(v_old - V[s]))
        if delta < theta:
            break

    # Extract greedy policy from the converged value function
    pi = np.zeros(nS, dtype=int)
    for s in range(nS):
        q = np.zeros(nA, dtype=np.float64)
        for a in range(nA):
            for (prob, ns, r, done) in P[s][a]:
                q[a] += prob * (r + gamma * (0.0 if done else V[ns]))
        pi[s] = int(np.argmax(q))

    return pi, V, iterations

In [6]:

pi_opt, V_opt, iters = value_iteration(gamma=gamma, theta=theta)
print(f'Converged in {iters} iterations')





# python
# Drop-in replacement for the V display section in `frosen_VALUE_ITERATION.ipynb`

rows_map = len(map_desc)
cols_map = len(map_desc[0]) if rows_map > 0 else 0


print('\nOptimal V (grid):')
if rows_map * cols_map == nS and rows_map > 0:
    V_grid = V_opt.reshape(rows_map, cols_map)
    for r in range(rows_map):
        print(' '.join(f'{V_grid[r, c]:>6.3f}' for c in range(cols_map)))
else:
    per_row = max(6, int(np.sqrt(nS)))
    for i in range(0, nS, per_row):
        print(' '.join(f'{v:>6.3f}' for v in V_opt[i:i+per_row]))


Converged in 11 iterations

Optimal V (grid):
 0.904  0.914  0.923  0.000  0.941  0.951
 0.000  0.923  0.932  0.941  0.951  0.961
 0.904  0.914  0.000  0.951  0.961  0.970
 0.000  0.000  0.990  0.000  0.000  0.980
 0.980  0.990  1.000  0.000  1.000  0.990


In [7]:
arrow_map = {0: '←', 1: '↓', 2: '→', 3: '↑'}

# Derive shape
rows_map = len(map_desc)
cols_map = len(map_desc[0]) if rows_map > 0 else 0

print("Optimal Policy:")
arrows = [arrow_map[a] for a in pi_opt]
if rows_map * cols_map == nS and rows_map > 0:
    grid = np.array(arrows).reshape(rows_map, cols_map)
    for r in range(rows_map):
        print(' '.join(grid[r]))
else:
    print(' '.join(arrows))

Optimal Policy:
→ ↓ ↓ ← ↓ ↓
← → → ↓ ↓ ↓
→ ↑ ← → → ↓
← ← ↓ ← ← ↓
→ → → ← ← ←


In [8]:
def run_episode(env, pi):
    obs, info = env.reset()
    terminated = False
    truncated = False
    total_reward = 0.0
    steps = 0
    while not (terminated or truncated):
        a = int(pi[obs])
        obs, r, terminated, truncated, info = env.step(a)
        total_reward += r
        steps += 1
    return total_reward, steps, terminated, truncated

In [9]:
tr, steps, term, trunc = run_episode(env, pi_opt)
print(f'\nEpisode -> reward: {tr}, steps: {steps}, terminated: {term}, truncated: {trunc}')

# Keep window visible briefly and pump events to avoid 'Not Responding'
try:
    import pygame
    for _ in range(100):  # ~3 seconds
        pygame.event.pump()
        time.sleep(0.03)
except Exception:
    time.sleep(3.0)

env.close()


C:\Users\LENOVO i7\AppData\Local\Programs\Python\Python312\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists



Episode -> reward: 1.0, steps: 11, terminated: True, truncated: False
